In [ ]:
from liblet import Tree, Grammar, Derivation, ProductionGraph, side_by_side

# Fare pulizia nelle CF

## Una nozione di chiusura

Applicare il decoratore [@closure](https://liblet.readthedocs.io/en/latest/api.html#liblet.decorators.closure) ad una funzione `f`  produce una funzione `F` che corrisponde all'applicazione di `f` iterata sino a quando l'argomento ed il valore restituito da quest'ultima coincidono.

In [ ]:
from liblet import closure 

@closure
def add_smaller(S):
  return S | {x - 1 for x in S if x > 0}

In [ ]:
add_smaller({3, 5})

## Eliminare simboli non produttivi e/o irraggiungibili

In [ ]:
# fig 2.27, pag. 49

G = Grammar.from_string("""
S -> A B | D E
A -> a
B -> b C
C -> c
D -> d F 
E -> e 
F -> f D
""")
G

In [ ]:
def find_productive(G):

  @closure
  def find(prod):
    return prod | {A for A, α in G.P if set(α) <= prod}

  return find(G.T)

find_productive(G)

In [ ]:
from liblet import union_of

def find_reachable(G):

  @closure
  def find(reach):
    return reach | union_of(set(α) for A, α in G.P if A in reach)

  return find({G.S})

find_reachable(G)

In [ ]:
def remove_unproductive_unreachable(G):
    Gp = G.restrict_to(find_productive(G))
    return Gp.restrict_to(find_reachable(Gp))

remove_unproductive_unreachable(G)

# Derivazioni e alberi di parsing

In [ ]:
# una grammatica banale per il linguaggio {'ab'}

G_ab = Grammar.from_string("""
S -> A B
A -> a
B -> b
""")
G_ab

In [ ]:
# due possibili derivazioni

ab_0 = Derivation(G_ab).step([
  (0, 0), (2, 1), (1, 0)
])
ab_1 = Derivation(G_ab).step([
  (0, 0), (1, 0), (2, 1)
])

ab_0, ab_1

In [ ]:
# ma a ben guardare lo stesso albero…

side_by_side(
  ProductionGraph(ab_0),
  ProductionGraph(ab_1),
)

## Dalla derivazione all'albero di parsing

La seguente procedura memorizza in `tree` l'albero di derivazione e in `frontier` la sua frontiera, corrispondente alla forma sentenziale a cui è giunta la derivazione (di passo in passo) come una lista di alberi annotati. 

Ciascun nodo dell'albero ha due etichette: `Symbol` che si riferisce ad uno dei simboli della grammatica e `nprord` pari ad un numero di produzione. I figli di ciascun nodo dell'albero hanno i simboli contenuti nel lato destro della produzione di numero `nprord`.

Gli alberi vengono "completati" man mano che la procedura elabora i passi della derivazione; al termine le foglie degli alberi saranno simboli terminali (e `nprord` sarà convenzionalmente definito come `None`).

In [ ]:
def derivation_to_parsetree(d):

    # questa variabile si riferià all'albero di derivazione di d
    # inizialmente contiene l'albero annotato col simbolo di partenza
    tree = Tree({'Symbol': d.G.S, 'nprod': None})

    # all'inizio la forma sentenziale è data da tale albero
    frontier = [tree, ] 

    for nprod, pos in d.steps():

        # i figli sono dati dal lato destro d.G.P[prod].rhs
        children = [Tree({'Symbol': X, 'nprod': None}) for X in d.G.P[nprod].rhs]

        # l'albero è dato dalla posizione in cui è applicata la produzione
        curr = frontier[pos]

        # si aggiornano l'albero e la forma sentenziale
        curr.root['nprod'] = nprod
        curr.children = children
        frontier = frontier[:pos] + children + frontier[pos + 1:]

    return tree

In [ ]:
pt_0 = derivation_to_parsetree(ab_0)
pt_0

In [ ]:
# l'albero di parsing prodotto da liblet, e da derivation_to_parsetree

side_by_side(
  ProductionGraph(ab_0),
  Tree.from_lol(pt_0)
)

## Dall'albero alle derivazioni (left- e right-most)

Le derivazioni leftmost e rightmost si possono ottenere, rispettivamente, tramite una visita in preordine e postordine dell'albero di parsing (la cui implementazoine è lasciata come esercizio)

## Le ε-produzioni

In alcuni casi sono comode produzioni della forma $A\to \epsilon$ (chiamate ε-produzioni). Nella costruzione dell'albero dalla derivazione richiedono però un po' di attenzione…

Consideriamo ad esempio il caso del [linguaggio di Dyck](https://en.wikipedia.org/wiki/Dyck_language)

In [ ]:
G_dyck = Grammar.from_string('S -> S S | ( S ) | ε')

In [ ]:
# una derivazione leftmost di '(())'

dyck = 0, 1, 1, 2, 2

deriv = Derivation(G_dyck).leftmost(dyck)
deriv

In [ ]:
# la parola 
''.join(deriv.sentential_form())

In [ ]:
# l'albero 

ProductionGraph(deriv)

Applicando la funzione `derivation_to_parsetree` si nota però un errore: sembra che da `)` derivi ε!

In [ ]:
derivation_to_parsetree(deriv) 

Si lascia come esercizio correggere la funzione `derivation_to_parsetree` in modo che funzioni correttamente anche nel caso di ε-produzioni.

# Ambiguità


## È sempre rilevante? 

Pensiamo a grammatiche della forma `E -> E op E | n` dove `op` sia un operatore binario, definiamo una funzione ricorsiva che, dato un albero di parsing e il valore di `n`, valuti l'espressione aritmetica corrispondente.

In [ ]:
from operator import add, sub, mul

DT = {
  '+': add,
  '-': sub,
  '*': mul
}

def eval_tree(tree, n):
  root, *children = tree
  if root['Symbol'] == 'n': return n
  if root['Symbol'] == 'E' and len(children) == 3:
    left, op, right = children
    return DT[op.root['Symbol']](eval_tree(left, n), eval_tree(right, n))
  else: return eval_tree(children[0], n)

### Il caso "spurio"

In [ ]:
# Costruzione di una derivazione leftomst

def lmd(G, leftmost_steps):
  return Derivation(G).leftmost(leftmost_steps)

In [ ]:
G_as = Grammar.from_string('E -> E + E | n')

# due derivazioni di n + n + n

lmd0, lmd1 = lmd(G_as, [0, 1, 0, 1, 1]), lmd(G_as, [0, 0, 1, 1, 1])

side_by_side(ProductionGraph(lmd0), ProductionGraph(lmd1))

In [ ]:
# 1 + 1 + 1, comunque parsato, vale 3

eval_tree(derivation_to_parsetree(lmd0), 1), eval_tree(derivation_to_parsetree(lmd1), 1)

### I casi "essenziali"

Iniziamo dall'associatività/commutatività

In [ ]:
G_ae = Grammar.from_string('E -> E - E | n')

# due derivazioni di n - n - n

lmd0, lmd1 = lmd(G_ae, [0, 1, 0, 1, 1]), lmd(G_ae, [0, 0, 1, 1, 1])

side_by_side(ProductionGraph(lmd0), ProductionGraph(lmd1))

In [ ]:
# 1 - (1 - 1) e (1 - 1) - 1 hanno valori diversi!

eval_tree(derivation_to_parsetree(lmd0), 1), eval_tree(derivation_to_parsetree(lmd1), 1)

Consideriamo la *precedenza* 

In [ ]:
G_perr = Grammar.from_string("""
E -> E + E | E * E | n
""")

# n + n * n

lmd0, lmd1 = lmd(G_perr, [1, 0, 2, 2, 2]), lmd(G_perr, [0, 2, 1, 2, 2])

side_by_side(ProductionGraph(lmd0), ProductionGraph(lmd1))

In [ ]:
# ((2 + 2) * 2) e 2 + (2 * 2) hanno valori diversi!

eval_tree(derivation_to_parsetree(lmd0), 2), eval_tree(derivation_to_parsetree(lmd1), 2)

Per finire, un esempio non collegato alle *espressioni*, ma al *dangling else*

In [ ]:
G_iferr = Grammar.from_string("""
S -> if C then S | if C then S else S | stm
C -> cond
""")

In [ ]:
# if cond the if cont then stm else stm

lmd0, lmd1 = lmd(G_iferr, [0, 3, 1, 3, 2, 2]), lmd(G_iferr, [1, 3, 0, 3, 2, 2])

side_by_side(ProductionGraph(lmd0), ProductionGraph(lmd1))

## Possibili soluzioni

Un simbolo in più, ricorsione *sinistra* o *destra* e relazione con l'*associatività*. Iniziamo con l'associatibità a sinistra.

In [ ]:
G_la = Grammar.from_string("""
E -> E + E' | E'
E' -> n 
""")
G_la

In [ ]:
ProductionGraph(lmd(G_la, [0, 0, 1, 2, 2, 2]))

E quindi consideriamo quella a destra.

In [ ]:
G_ra = Grammar.from_string("""
E -> E' ^ E | E'
E' -> n 
""")
G_ra

In [ ]:
ProductionGraph(lmd(G_ra, [0, 2, 0, 2, 1, 2]))

Per la *precedenza* occorre aggiungere più di un simbolo…

In [ ]:
G_p = Grammar.from_string("""
E -> E + P | P
P -> P * F | F
F -> n
""")

In [ ]:
ProductionGraph(lmd(G_p, [0, 1, 3, 4, 2, 3, 4, 4]))

Il caso del *dangling else*

In [ ]:
G_if = Grammar.from_string("""
S -> M | U
M -> if C then M else M | stm
U -> if C then M else U | if C then S
C -> cond
""")

In [ ]:
ProductionGraph(lmd(G_if, [1, 5, 6, 0, 2, 6, 3, 3]))